In [2]:
from selenium import webdriver
import re
import pandas as pd
import json
import datetime
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
executable_path = "/Users/justinsech/opt/anaconda3/lib/python3.8/site-packages/chromedriver_binary/chromedriver"
browser = webdriver.Chrome(executable_path)
browser.implicitly_wait(5)

# Tokboard

In [81]:
url="https://tokboard.com/months"


months = ["January", 'February', "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"]
years = ["2018", "2019", "2020", "2021"]

record = {}
for y in years:
    for m in months:
        if f"{m}-{y}" == "May-2021":
            break
        url = f"https://tokboard.com/months/{m}-{y}"
        browser.get(url)
        page = {}
        songs = browser.find_elements_by_class_name("song")
        for i, s in enumerate(songs):
            artist = s.find_element_by_class_name("artist").get_attribute("innerHTML")
            title = s.find_element_by_class_name("title").find_element_by_tag_name("a").get_attribute("innerHTML")
            
            views = s.find_element_by_class_name("views").get_attribute("innerHTML")
            views = re.findall(r"\d+\.?\d{0,1}[BMK]", views)[0]
            
            videos = s.find_element_by_class_name("videos").get_attribute("innerHTML")
            videos = re.findall(r"\d+\.?\d{0,1}[BMK]?", videos)[0]
            
            change = s.find_element_by_class_name("change").find_element_by_tag_name("a").get_attribute("innerHTML")
            page[i+1] = {"artist": artist, "title":title, "views": views, "videos": videos}  
        record[f"{m}-{y}"] = page

In [118]:
with open("./data/tokboard_record.json", "w") as f:
    json.dump(record, f)

# Spotify

### Viral 50

In [160]:
url = "https://spotifycharts.com/viral/global/weekly/"
browser.get(url)

# Compile the list of dates we need to download Charts for
stop_date = '2018-07-26--2018-07-26'
dates = []
for item in browser.find_element_by_xpath('//*[@id="content"]/div/div/div/span/div[1]/div/div/div/div[3]').find_elements_by_tag_name("li"):
    date = item.get_attribute("data-value")
    dates.append(date)
    if date == stop_date:
        break

In [193]:
# Scrape the Viral 50 chart for each week in our list of dates
def viral50(region):
    url = f"https://spotifycharts.com/viral/{region}/weekly/"
    viral_50 = {}
    for date in dates:
        cur = {}
        browser.get(f"{url}{date}")
        tracks = browser.find_elements_by_class_name("chart-table-track")[1:] # first table-track is HEADER
        for i, t in enumerate(tracks):
            title = t.find_element_by_tag_name("strong").get_attribute("innerHTML")
            artist = t.find_element_by_tag_name("span").get_attribute("innerHTML")
            cur[i] = {"title": title, "artist": artist}
        viral_50[date] = cur
    return viral_50

In [174]:
global_viral_50 = viral50("global")
with open("./data/global_viral_50.json", "w") as f:
    json.dump(global_viral_50, f)

In [ ]:
usa_viral_50 = viral50("us")
with open("./data/usa_viral_50.json", "w") as f:
    json.dump(usa_viral_50, f)

### Top 200

In [180]:
url = "https://spotifycharts.com/regional/global/weekly/"
browser.get(url)

# Compile the list of dates we need to download Charts for
stop_date = '2018-07-20--2018-07-27'
dates = []
for item in browser.find_element_by_xpath('//*[@id="content"]/div/div/div/span/div[1]/div/div/div/div[3]').find_elements_by_tag_name("li"):
    date = item.get_attribute("data-value")
    dates.append(date)
    if date == stop_date:
        break

In [196]:
# Scrape the Top 200 chart for each week in our list of dates
def top200(region):
    url = f"https://spotifycharts.com/regional/{region}/weekly/"
    top_200 = {}
    for date in dates:
        cur = {}
        browser.get(f"{url}{date}")
        tracks = browser.find_elements_by_class_name("chart-table-track")[1:] # first table-track is HEADER
        streams = browser.find_elements_by_class_name("chart-table-streams")[1:] # first table-track is HEADER
        for i, (t, s) in enumerate(zip(tracks, streams)):
            title = t.find_element_by_tag_name("strong").get_attribute("innerHTML")
            artist = t.find_element_by_tag_name("span").get_attribute("innerHTML")
            plays = s.get_attribute("innerHTML")
            cur[i] = {"title": title, "artist": artist, "streams": plays}
        top_200[date] = cur
    return top_200

In [197]:
global_top_200 = top200("global")

In [198]:
with open("./data/global_top_200.json", "w") as f:
    json.dump(global_top_200, f)

In [199]:
usa_top_200 = top200("us")
with open("./data/usa_top_200.json", "w") as f:
    json.dump(usa_top_200, f)

### Get Artist Info

In [3]:
CLIENT_ID = "69200cfc38774a4394c7bc17911d0c08"
CLIENT_SECRET = "6c5f36f3da3e4e699ebc7ba59d3d9792"
auth_manager= spotipy.oauth2.SpotifyClientCredentials(client_id=CLIENT_ID, client_secret=CLIENT_SECRET)#, redirect_uri=REDIRECT_URI, scope='playlist-modify-public, playlist-modify-private')
sp = spotipy.Spotify(client_credentials_manager=auth_manager)

In [29]:
with open("./data/global_viral_50.json", "r") as f:
    charts = json.load(f)

In [87]:
# split individual artists 
artists = set()
for chart in charts:
    week = charts[chart]
    for song in week:
        temp = week[song]['artist']

        # remove "by " from atrist name
        temp = temp[3:]
        
        # clean text
        temp = re.sub(r"&amp;", "&", temp)
        
        # add to set
        for name in temp.split(", "):
            artists.add(name.lower())          

In [108]:
artist_info = {}
for name in artists:
    results = sp.search(q="artist:"+name, type="artist")['artists']['items']
    if len(results) == 0:
        continue
    results = results[0]
    uri = results['uri']
    pop = results['popularity']
    followers = results['followers']['total']
    genres = results['genres']
    related = []
    for r in sp.artist_related_artists(uri)['artists']:
        related.append((r['name'], r['uri'])) 
    artist_info[name] = {"uri": uri, "popularity": pop, "followers": followers, "genres": genres, "related": related}

ReadTimeout: HTTPSConnectionPool(host='api.spotify.com', port=443): Read timed out. (read timeout=5)

dict_keys(['明美圭一', '', 'kumaar', 'rita lee', 'wrongonyou', 'seraphine', 'shinsky x dima', 'jorja smith', 'lecrae', 'iu', 'robbz x brookz', 'kacey musgraves', 'dj khaled', 'stunna 4 vegas', 'bizarrap', 'maikel delacalle', 'aga b', '$uicideboy$', 'minette martin', 'dream', 'noah cyrus', 'nio garcia', 'slatin', 'yofrangel', 'sueco', 'powfu', 'kaka', 'seungri', 'death cab for cutie', 'polo pi', 'mc dricka', 'chris viz', 'the plug', 'salatiel', 'king', 'greekazo', 'slushii', 'canbay', 'smokepurpp', 'ximena sariñana', 'wisin', 'yola', 'alan walker', 'savage ga$p', "why don't we", 'mc lemos', 'pablo chill-e', 'nate smith'])

# Bilboard

In [4]:
# Compile the list of dates we need to download Charts for
base = datetime.datetime.strptime("2018-07-28", "%Y-%m-%d")
date_list = [(base + 7*datetime.timedelta(days=x)).strftime("%Y-%m-%d") for x in range(144)]

### Hot 100

In [73]:
# Scrape the Viral 50 chart for each week in our list of dates
url = "https://www.billboard.com/charts/hot-100"
hot_100 = {}
for date in date_list:
    cur = {}
    browser.get(f"{url}/{date}")
    chart = browser.find_elements_by_class_name("chart-list__element")
    for i, row in enumerate(chart):
        song = row.find_element_by_class_name("chart-element__information__song").get_attribute("innerHTML")
        artist = row.find_element_by_class_name("chart-element__information__artist").get_attribute("innerHTML")
        last = row.find_element_by_class_name("text--last").get_attribute("innerHTML")
        peak = row.find_element_by_class_name("text--peak").get_attribute("innerHTML")
        weeks = row.find_element_by_class_name("text--week").get_attribute("innerHTML")
        peak = re.findall(r"\d+", peak)[0]
        weeks = re.findall(r"\d+", weeks)[0]
        last = re.findall(r"\d+", last)
        if last == []:
            last = "-1"
        else:
            last = last[0]

        cur[i] = {"song": song, "artist": artist, "last": int(last), "peak": int(peak), "weeks": int(weeks)}
    hot_100[date] = cur

In [74]:
with open("./data/bilboard_hot_100.json", "w") as f:
    json.dump(hot_100, f)

### BIlboard 200

In [84]:
# Scrape the Viral 50 chart for each week in our list of dates
url = "https://www.billboard.com/charts/billboard-200"
top_200 = {}
for date in date_list:
    cur = {}
    browser.get(f"{url}/{date}")
    chart = browser.find_elements_by_class_name("chart-list__element")
    for i, row in enumerate(chart):
        song = row.find_element_by_class_name("chart-element__information__song").get_attribute("innerHTML")
        artist = row.find_element_by_class_name("chart-element__information__artist").get_attribute("innerHTML")
        last = row.find_element_by_class_name("text--last").get_attribute("innerHTML")
        peak = row.find_element_by_class_name("text--peak").get_attribute("innerHTML")
        weeks = row.find_element_by_class_name("text--week").get_attribute("innerHTML")
        peak = re.findall(r"\d+", peak)[0]
        weeks = re.findall(r"\d+", weeks)[0]
        last = re.findall(r"\d+", last)
        if last == []:
            last = "-1"
        else:
            last = last[0]

        cur[i] = {"song": song, "artist": artist, "last": int(last), "peak": int(peak), "weeks": int(weeks)}
    top_200[date] = cur

In [85]:
with open("./data/bilboard_200.json", "w") as f:
    json.dump(top_200, f)